<a href="https://colab.research.google.com/github/ArthurCTLin/Workbook/blob/main/Introduction%20to%20Numerical%20Analysis/Lecture304_hw_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Connect to TPU
* Edit -> NoteBook settings
* Select TPU from the Hardware Accelerator drop-down
* Run the following script to check the connect situation

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.1
Running on TPU  ['10.42.191.146:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.42.191.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.42.191.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### Connect to google drive for loading data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls "/content/drive/MyDrive/ColabNotebooks/"

font_data.npy  l304practice.npz  Lecture304_hw_01.ipynb


In [7]:
import numpy as np
import matplotlib.pyplot as plt

data = np.load('/content/drive/MyDrive/ColabNotebooks/l304practice.npz')
x_train = data['x_train']
x_train = x_train/127.5-1.

latent_size = 128
img_shape = (48,48)

from keras.layers import Input, Dense, Reshape
from keras.layers import BatchNormalization, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

discriminator = Sequential()
discriminator.add(Reshape((np.prod(img_shape),),input_shape=img_shape))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU())
discriminator.add(Dense(256))
discriminator.add(LeakyReLU())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(0.0002, 0.5),
                      metrics=['accuracy'])

generator = Sequential()
generator.add(Dense(256, input_dim=latent_size))
generator.add(LeakyReLU())
generator.add(BatchNormalization())
generator.add(Dense(512))
generator.add(LeakyReLU())
generator.add(BatchNormalization())
generator.add(Dense(1024))
generator.add(LeakyReLU())
generator.add(BatchNormalization())
generator.add(Dense(np.prod(img_shape), activation='tanh'))
generator.add(Reshape(img_shape))

noise = Input(shape=(latent_size,))
img = generator(noise)
discriminator.trainable = False
validity = discriminator(img)
combined = Model(noise, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

noise_test = np.random.randn(100, latent_size)
batch_size = 32
for epoch in range(20001):
    
    imgs_real = x_train[np.random.randint(0, len(x_train), batch_size)]
    
    noise = np.random.randn(batch_size, latent_size)
    imgs_fake = generator.predict(noise)

    dis_loss_real = discriminator.train_on_batch(imgs_real, np.ones((batch_size,1)))
    dis_loss_fake = discriminator.train_on_batch(imgs_fake, np.zeros((batch_size,1)))
    dis_loss = np.add(dis_loss_real,dis_loss_fake)*0.5

    noise = np.random.randn(batch_size, latent_size)
    gen_loss = combined.train_on_batch(noise, np.ones((batch_size,1)))
                                       
    print("Epoch: %d, discriminator(loss: %.3f, acc.: %.2f%%), generator(loss: %.3f)" %
          (epoch, dis_loss[0], dis_loss[1]*100., gen_loss))

    if epoch % 400 == 0:
        imgs_fake = generator.predict(noise_test)
                                       
        fig = plt.figure(figsize=(10,10), dpi=80)
        plt.subplots_adjust(0.05,0.05,0.95,0.95,0.1,0.1)
        for i in range(100):
            plt.subplot(10,10,i+1)
            plt.axis('off')
            plt.imshow(imgs_fake[i], cmap='Greys')
        fig.suptitle('epoch: %05d' % epoch, color='Blue', fontsize=16)
        fig.savefig("/content/drive/MyDrive/ColabNotebooks/%05d.png" % epoch)
        plt.close()

串流輸出內容已截斷至最後 5000 行。
Epoch: 15001, discriminator(loss: 0.246, acc.: 89.06%), generator(loss: 4.142)
Epoch: 15002, discriminator(loss: 0.225, acc.: 89.06%), generator(loss: 3.799)
Epoch: 15003, discriminator(loss: 0.268, acc.: 87.50%), generator(loss: 4.724)
Epoch: 15004, discriminator(loss: 0.238, acc.: 90.62%), generator(loss: 3.790)
Epoch: 15005, discriminator(loss: 0.291, acc.: 89.06%), generator(loss: 3.505)
Epoch: 15006, discriminator(loss: 0.201, acc.: 93.75%), generator(loss: 3.894)
Epoch: 15007, discriminator(loss: 0.145, acc.: 93.75%), generator(loss: 4.180)
Epoch: 15008, discriminator(loss: 0.149, acc.: 95.31%), generator(loss: 3.995)
Epoch: 15009, discriminator(loss: 0.107, acc.: 95.31%), generator(loss: 3.303)
Epoch: 15010, discriminator(loss: 0.262, acc.: 90.62%), generator(loss: 3.530)
Epoch: 15011, discriminator(loss: 0.217, acc.: 90.62%), generator(loss: 3.628)
Epoch: 15012, discriminator(loss: 0.112, acc.: 95.31%), generator(loss: 3.481)
Epoch: 15013, discriminator(los